In [1]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import gensim

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
import string
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
!pip install torchtext==0.4

     |████████████████████████████████| 61kB 2.0MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
import torch
from torchtext import data, datasets, vocab, utils
import torch.nn as nn
import torchsummary

In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fake_News_Detection/dataset/data_liar_plus.csv', sep=';')

In [0]:
le_multi = LabelEncoder()
df.loc[:, 'label'] = le_multi.fit_transform(df.label)

**Удаляю пунктуацию и понижаю размерность**

In [0]:
df.statement = df.statement.apply(lambda x: x.lower().strip())
df.statement = df.statement.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [0]:
train_index = df.query("data_type == 'train'").index
val_index = df.query("data_type == 'val'").index
test_index = df.query("data_type == 'test'").index

In [0]:
y_train_multi = df.loc[train_index, 'label']
y_val_multi = df.loc[val_index, 'label']
y_test_multi = df.loc[test_index, 'label']

y_train_binary = df.loc[train_index, 'is_fake']
y_val_binary = df.loc[val_index, 'is_fake']
y_test_binary = df.loc[test_index, 'is_fake']

In [0]:
train_sentences = df.loc[train_index].statement.values
val_sentences = df.loc[val_index].statement.values
test_sentences = df.loc[test_index].statement.values

In [0]:
df.loc[train_index][['statement', 'is_fake']].to_csv('train_data.csv', index=False)
df.loc[val_index][['statement', 'is_fake']].to_csv('val_data.csv', index=False)
df.loc[test_index][['statement', 'is_fake']].to_csv('test_data.csv', index=False)

# Baseline

##Majority vote model (baseline)

In [0]:
def get_metrics(true, predicted, binary=True):
  if binary:
    return pd.DataFrame({"Accuracy": [round(accuracy_score(true, predicted), 4)],
  "Precision": [round(precision_score(true, predicted), 4)],
  "Recall": [round(recall_score(true, predicted), 4)],
  "F1": [round(f1_score(true, predicted), 4)]})
  else:
    return pd.DataFrame({"Accuracy": [round(accuracy_score(true, predicted), 4)],
  "Precision": [round(precision_score(true, predicted, average='macro'), 4)],
  "Recall": [round(recall_score(true, predicted, average='macro'), 4)],
  "F1": [round(f1_score(true, predicted, average='macro'), 4)]})

### Multiclass classification

In [0]:
multi_popular = y_train_multi.value_counts().index[0]

In [14]:
get_metrics(y_test_multi, [multi_popular]*len(y_test_multi), binary=False)

,Accuracy,Precision,Recall,F1
0,0.2092,0.0349,0.1667,0.0577


### Binary Classification

In [0]:
binary_popular = y_train_binary.value_counts().index[0]

In [16]:
get_metrics(y_test_binary, [binary_popular]*len(y_test_binary), binary=False)

,Accuracy,Precision,Recall,F1
0,0.5635,0.2818,0.5,0.3604


## TF-IDF LogisticRegression

### "Statement" model

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True, max_df=0.8)

In [0]:
X_train = tfidf.fit_transform(df.loc[train_index, 'statement'])
X_val = tfidf.transform(df.loc[val_index, 'statement'])
X_test = tfidf.transform(df.loc[test_index, 'statement'])

#### Multiclass classification

In [0]:
lr = LogisticRegression(max_iter=500)

In [21]:
lr.fit(X_train, y_train_multi)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
get_metrics(y_test_multi, lr.predict(X_test), binary=False)

,Accuracy,Precision,Recall,F1
0,0.2612,0.3321,0.2305,0.2232


#### Binary Classification 

In [0]:
lr = LogisticRegression(C=2)

In [24]:
lr.fit(X_train, y_train_binary)

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
get_metrics(y_test_binary, lr.predict(X_test))

,Accuracy,Precision,Recall,F1
0,0.6267,0.6081,0.4069,0.4875


## TF-IDF SVM

In [0]:
svm = SVC(kernel='linear')

In [27]:
svm.fit(X_train, y_train_binary)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [28]:
get_metrics(y_test_binary, svm.predict(X_test))

,Accuracy,Precision,Recall,F1
0,0.6251,0.5995,0.425,0.4974


# GoogleNews Embeddings

In [12]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2020-05-30 13:16:33--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G   113MB/s    in 17s     

2020-05-30 13:16:50 (95.1 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
embeddings_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
vocab = embeddings_model.vocab
embedding = embeddings_model.syn0

In [0]:
#vectors = vocab.Vectors(name='GoogleNews-vectors-negative300.bin.gz')

In [15]:
# всего слов в датасете
len(' '.join(df.statement.values).split())

228228

In [0]:
vocabulary = {}
for word in ' '.join(df.statement.values).split():
  if word in vocabulary.keys():
    vocabulary[word] += 1
  else:
    vocabulary[word] = 1

In [0]:
success = 0
word_cnt = 0
unknown_words = []
for word in vocabulary.keys():
  if word in vocab.keys():
    success += 1
    word_cnt += vocabulary[word]
  else:
    unknown_words.append(word)

In [18]:
print(f"Found embeddings for {success*100//len(vocabulary.keys())}% of vocabulary")
print(f"Found embeddings for {word_cnt*100//len(' '.join(df.statement.values).split())}% of all text")

Found embeddings for 77% of vocabulary
Found embeddings for 86% of all text


# "Statement" model. Data preprocessing

In [0]:
# Вот полная функция для фиксации сида:
# def seed_torch(seed=1029):
# random.seed(seed)
# os.environ['PYTHONHASHSEED'] = str(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

In [0]:
EMBED_SIZE = 300 
MAX_FEATURES = 13313 # 13313 все слова
MAXLEN = 66 # 66 - максимум слов в statement
BATCH_SIZE = 128

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
STATEMENT = data.Field(sequential=True)
TARGET = data.Field(sequential=False, use_vocab=False, dtype=torch.int64)
data_fields = [('statement', STATEMENT),
               ('is_fake', TARGET)]

In [0]:
train = data.TabularDataset(path='train_data.csv', format='csv', fields=data_fields, skip_header=True)
val = data.TabularDataset(path='val_data.csv', format='csv', fields=data_fields, skip_header=True)
test = data.TabularDataset(path='test_data.csv', format='csv', fields=data_fields, skip_header=True)

In [0]:
STATEMENT.build_vocab(train)

In [0]:
emb_mean, emb_std = embedding.mean(), embedding.std()

word_index = dict(STATEMENT.vocab.stoi).keys()
unknown_words = set()
nb_words = len(word_index)
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBED_SIZE))
for word in word_index:
      index = STATEMENT.vocab[word]
      if index != 0:
        try:
          embedding_vector = embeddings_model[word]
          embedding_matrix[index] = embedding_vector
        except KeyError:
          unknown_words.add(word)
      else:
        continue
        
embedding_matrix = torch.from_numpy(embedding_matrix)

In [33]:
vocab_size = len(STATEMENT.vocab.itos)
print(f"Vocabulary size {vocab_size}")
print(f"Unknown words {len(unknown_words)}")

Vocabulary size 13315
Unknown words 2864


In [0]:
STATEMENT.vocab.set_vectors(STATEMENT.vocab.stoi, embedding_matrix, EMBED_SIZE)

In [0]:
train_iter = data.BucketIterator(train,
                                batch_size=BATCH_SIZE,
                                sort_key= lambda x: len(x.statement),
                                repeat=False,
                                shuffle=True,
                                device=device)

val_iter = data.BucketIterator(val,
                                batch_size=BATCH_SIZE,
                                sort_key= lambda x: len(x.statement),
                                repeat=False,
                                sort = True,
                                device=device)

## Bi-LSTM (Statement)

In [0]:
output_size = 2
hidden_size = 150

In [0]:
class BILSTMClassifier(nn.Module):
    def __init__(self, vocab_size, batch_size, output_size, hidden_size, embedding_length):
        super(BILSTMClassifier, self).__init__()

        """
        Arguments
        ---------
        batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
        output_size : 3 = (normal, small, big)
        hidden_sie : Size of the hidden_state of the LSTM
        embedding_length : Embeddding dimension of FastText word embeddings
        
        """

        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.embedding_length = embedding_length
        self.vocab_size = vocab_size

        #self.embedding = nn.Embedding.from_pretrained(weights)
        self.embedding = nn.Embedding(vocab_size, embedding_length, padding_idx=1)
        self.embedding.weight = nn.Parameter(STATEMENT.vocab.vectors, requires_grad=True)

        self.lstm = nn.LSTM(embedding_length, hidden_size, bidirectional=True, num_layers=2)
        self.label = nn.Linear(hidden_size*2*2, output_size)

    def forward(self, input_sentence, batch_size=None):
        input = self.embedding(input_sentence)
        #print(input.shape)
        output, (final_hidden_state, final_cell_state) = self.lstm(input)
        final_output = self.label(torch.cat([final_hidden_state[i,:,:] for i in range(int(final_hidden_state.shape[0]))], dim=1))  # final_hidden_state.size() = (1, batch_size, hidden_size)

        return final_output

In [0]:
lstm_model = BILSTMClassifier(vocab_size, BATCH_SIZE, output_size, hidden_size, EMBED_SIZE).to(device)

In [0]:
import torch.optim as optim

In [0]:
optimizer = optim.Adam(lstm_model.parameters())
loss_func = nn.CrossEntropyLoss().to(device)

In [42]:
for epoch in range(5): 

    running_loss = 0
    train_acc = 0

    for el in train_iter:
        inputs, labels = el.statement.to(device), el.is_fake.to(device)
        optimizer.zero_grad()

        outputs = lstm_model(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        train_acc += (outputs.argmax(1) == labels).sum().item()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1} | Loss {running_loss / (len(train) / BATCH_SIZE)} | Accuracy {train_acc / len(train)}')


print('Finished Training')

Epoch 1 | Loss 0.6695748932963821 | Accuracy 0.586638015237351
Epoch 2 | Loss 0.6081281864310084 | Accuracy 0.675815588982223
Epoch 3 | Loss 0.49547669495845076 | Accuracy 0.7727095135768705
Epoch 4 | Loss 0.35269013781360314 | Accuracy 0.8486032428208634
Epoch 5 | Loss 0.23701901816837395 | Accuracy 0.9048642312951748
Finished Training
